# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Import

In [1]:
import pandas as pd
import numpy as np


# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
import pandas as pd

dtype_dict = {
    'bathrooms':float, 
    'waterfront':int, 
    'sqft_above':int, 
    'sqft_living15':float, 
    'grade':int, 
    'yr_renovated':int, 
    'price':float, 
    'bedrooms':float, 
    'zipcode':str, 
    'long':float, 
    'sqft_lot15':float, 
    'sqft_living':float, 
    'floors':float,
    'condition':int,
    'lat':float,
    'date':str,
    'sqft_basement':int,
    'yr_built':int,
    'id':str,
    'sqft_lot':int,
    'view':int
}

sales = pd.read_csv('week5/kc_house_data.csv', dtype=dtype_dict)

In [3]:
sales.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180.0,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340.0,5650.0
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570.0,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690.0,7639.0
2,5631500400,20150225T000000,180000.0,2.0,1.00,770.0,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720.0,8062.0
3,2487200875,20141209T000000,604000.0,4.0,3.00,1960.0,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360.0,5000.0
4,1954400510,20150218T000000,510000.0,3.0,2.00,1680.0,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800.0,7503.0


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [4]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [5]:
from sklearn import linear_model  # using scikit-learn

all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

model_all = linear_model.Lasso(alpha=5e2, normalize=True) # set parameters
model_all.fit(sales[all_features], sales['price']) # learn weights

Lasso(alpha=500.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [8]:
model_all = linear_model.Lasso(alpha=5e2, normalize=True) # set parameters
model_all.fit(sales[all_features], sales['price']) # learn weights

Lasso(alpha=500.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

Find what features had non-zero weight.

In [12]:
# filter(zip(all_features, model_all.coef_),
filter(lambda x: x[1] != 0, zip(all_features, model_all.coef_))

[('sqft_living', 134.43931395541438),
 ('view', 24750.00458560952),
 ('grade', 61749.103090708129)]

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [15]:
testing = pd.read_csv('week5/wk3_kc_house_test_data.csv', dtype=dtype_dict)
training = pd.read_csv('week5/wk3_kc_house_train_data.csv', dtype=dtype_dict)
validation = pd.read_csv('week5/wk3_kc_house_valid_data.csv', dtype=dtype_dict)

testing['sqft_living_sqrt'] = testing['sqft_living'].apply(sqrt)
testing['sqft_lot_sqrt'] = testing['sqft_lot'].apply(sqrt)
testing['bedrooms_square'] = testing['bedrooms']*testing['bedrooms']
testing['floors_square'] = testing['floors']*testing['floors']

training['sqft_living_sqrt'] = training['sqft_living'].apply(sqrt)
training['sqft_lot_sqrt'] = training['sqft_lot'].apply(sqrt)
training['bedrooms_square'] = training['bedrooms']*training['bedrooms']
training['floors_square'] = training['floors']*training['floors']

validation['sqft_living_sqrt'] = validation['sqft_living'].apply(sqrt)
validation['sqft_lot_sqrt'] = validation['sqft_lot'].apply(sqrt)
validation['bedrooms_square'] = validation['bedrooms']*validation['bedrooms']
validation['floors_square'] = validation['floors']*validation['floors']

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [34]:
import numpy as np
def RSS(y_true, y_predict):
    residual = y_true - y_predict
    return np.dot(residual, residual)


    
def train_model(x, y, l1_penalty):
    model = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    model.fit(x, y)
    return model

def model_rss(model, x, y):
    y_predict = model.predict(x)
    error = y - y_predict
    return np.dot(error, error)


def model_weights(model, features):
    coef = model.coef_
    weights = zip(features, model.coef_)
    weights.append(('(intercept)', model.intercept_))
    weights = filter(lambda x: x[1] != 0, weights)
    weights = dict(weights)
    return weights
    

In [39]:
l1_penalty = 1e5

from pprint import pprint
for l1_penalty in np.logspace(1, 7, num=13):
    model = train_model(training[all_features], training['price'], l1_penalty)
    rss = model_rss(model, validation[all_features], validation['price'])
    weights = model_weights(model, all_features)
    print(l1_penalty, rss, end='\t ')
    print(np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_))
    

10.0 3.982133273e+14	 15
31.6227766017 3.99041900253e+14	 15
100.0 4.29791604073e+14	 11
316.227766017 4.63739831045e+14	 6
1000.0 6.45898733634e+14	 4
3162.27766017 1.22250685943e+15	 1
10000.0 1.22250685943e+15	 1
31622.7766017 1.22250685943e+15	 1
100000.0 1.22250685943e+15	 1
316227.766017 1.22250685943e+15	 1
1000000.0 1.22250685943e+15	 1
3162277.66017 1.22250685943e+15	 1
10000000.0 1.22250685943e+15	 1


*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [40]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [12]:
l1_penalty_values = np.logspace(1, 4, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [41]:

for l1_penalty in np.logspace(1, 4, num=20):
    model = train_model(training[all_features], training['price'], l1_penalty)
    rss = model_rss(model, validation[all_features], validation['price'])
    weights = model_weights(model, all_features)
    print(l1_penalty, rss, end='\t ')
    print(np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_))

10.0 3.982133273e+14	 15
14.3844988829 3.96831833944e+14	 15
20.6913808111 3.96210901853e+14	 15
29.7635144163 3.98215534575e+14	 15
42.8133239872 4.0687725852e+14	 13
61.5848211066 4.24647490491e+14	 12
88.586679041 4.27906308934e+14	 11
127.42749857 4.35374677103e+14	 10
183.298071083 4.43107216261e+14	 7
263.665089873 4.54176669663e+14	 6
379.269019073 4.78132980832e+14	 6
545.559478117 5.31397181867e+14	 6
784.759970351 5.94043306274e+14	 5
1128.83789168 6.74059169986e+14	 3
1623.77673919 8.02609410823e+14	 3
2335.72146909 1.06125525287e+15	 2
3359.81828628 1.22250685943e+15	 1
4832.93023857 1.22250685943e+15	 1
6951.92796178 1.22250685943e+15	 1
10000.0 1.22250685943e+15	 1


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

In [42]:
l1_penalty_min = 127
l1_penalty_max = 264

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [25]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [47]:

best_model = None
min_rss = 1e20

for l1_penalty in np.linspace(l1_penalty_min, l1_penalty_max, 20):
    model = train_model(training[all_features], training['price'], l1_penalty)
    rss = model_rss(model, validation[all_features], validation['price'])
    nonzeros = np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)
    if rss < min_rss and nonzeros == 7:
        min_rss = rss
        best_model = model
    weights = model_weights(model, all_features)
    print(l1_penalty, rss, nonzeros)

127.0 4.35280594306e+14 10
134.210526316 4.36918200309e+14 10
141.421052632 4.38193198929e+14 8
148.631578947 4.39118116501e+14 8
155.842105263 4.4001087979e+14 7
163.052631579 4.40753343343e+14 7
170.263157895 4.41545504585e+14 7
177.473684211 4.42388738846e+14 7
184.684210526 4.43283125273e+14 7
191.894736842 4.44230844529e+14 7
199.105263158 4.45227024002e+14 7
206.315789474 4.46270492654e+14 6
213.526315789 4.47119434939e+14 6
220.736842105 4.48010086061e+14 6
227.947368421 4.48942450569e+14 6
235.157894737 4.49916528461e+14 6
242.368421053 4.50932323385e+14 6
249.578947368 4.51990484738e+14 6
256.789473684 4.53089679757e+14 6
264.0 4.54230585503e+14 6


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [50]:
pprint(model_weights(best_model, all_features))


{'(intercept)': 4425723.965393858,
 'bathrooms': 10667.085435411589,
 'grade': 116281.46412928487,
 'sqft_living': 163.36252656145871,
 'view': 41964.588841716642,
 'waterfront': 506664.43257622316,
 'yr_built': -2614.1794842274999}
